In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("NEW_DATA/train.csv")
test = pd.read_csv("NEW_DATA/test.csv")

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28563 entries, 0 to 28562
Data columns (total 64 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_id             28563 non-null  object 
 1   units                28563 non-null  int64  
 2   OP_latitude          28563 non-null  float64
 3   OP_longitude         28563 non-null  float64
 4   LH_latitude          28298 non-null  float64
 5   LH_longitude         28298 non-null  float64
 6   customer_latitude    28563 non-null  float64
 7   customer_longitude   28563 non-null  float64
 8   distance1            28563 non-null  float64
 9   distance2            28563 non-null  float64
 10  weight               28563 non-null  float64
 11  OP_Athens            28563 non-null  int64  
 12  OP_Barcelona         28563 non-null  int64  
 13  OP_Rotterdam         28563 non-null  int64  
 14  3pl_v_001            28563 non-null  int64  
 15  3pl_v_002            28563 non-null 

In [4]:
delayed = train[train.late_order == True]
non_delayed = train[train.late_order == False]
shuffled = list(non_delayed.index)
random.shuffle(shuffled)
items = len(shuffled)//3

trucation_1 = shuffled[:items]
trucation_2 = shuffled[items:items*2]
trucation_3 = shuffled[items*2:]
df_part1 = non_delayed[non_delayed.index.isin(trucation_1)]
df_part2 = non_delayed[non_delayed.index.isin(trucation_2)]
df_part3 = non_delayed[non_delayed.index.isin(trucation_3)]
print(len(df_part1), len(df_part2), len(df_part3))
print(len(delayed))

29040 29040 29040
27156


In [5]:
df1 = pd.concat([df_part1, delayed])
df2 = pd.concat([df_part2, delayed])
df3 = pd.concat([df_part3, delayed])

df1.dropna(inplace=True)
df2.dropna(inplace=True)
df3.dropna(inplace=True)

In [6]:
train1, test1 = train_test_split(df1, test_size=0.2)

In [7]:
train1_x = train1.drop(columns=["late_order"])
train1_y = train1["late_order"]
test1_x = test1.drop(columns=["late_order"])
test1_y = test1["late_order"]

In [8]:
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

2022-11-13 06:33:27.026045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 06:33:27.189911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 06:33:27.189928: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-13 06:33:27.219382: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 06:33:28.104775: W tensorflow/stream_executor/platform/de

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, input_shape = (train1_x.shape[1],), activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

2022-11-13 06:33:28.814285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-13 06:33:28.814314: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-13 06:33:28.814339: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Nasar-Ubuntu-HP): /proc/driver/nvidia/version does not exist
2022-11-13 06:33:28.814535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.fit(x=train1_x,y=train1_y, epochs = 30, validation_data=(test1_x, test1_y))

Epoch 1/30


2022-11-13 06:33:29.138405: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22432032 exceeds 10% of free system memory.


1391/1391 [==============================] - 5s 3ms/step - loss: 0.7239 - accuracy: 0.5943 - val_loss: 0.6363 - val_accuracy: 0.6368
Epoch 2/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.6308 - accuracy: 0.6427 - val_loss: 0.6355 - val_accuracy: 0.6368
Epoch 3/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.6033 - accuracy: 0.6674 - val_loss: 0.6018 - val_accuracy: 0.6701
Epoch 4/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5907 - accuracy: 0.6789 - val_loss: 0.5821 - val_accuracy: 0.6882
Epoch 5/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5847 - accuracy: 0.6836 - val_loss: 0.5713 - val_accuracy: 0.6977
Epoch 6/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5794 - accuracy: 0.6886 - val_loss: 0.5727 - val_accuracy: 0.6982
Epoch 7/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5755 - accuracy: 0.6926 - val_loss: 0.5762 - val_accuracy: 0.6888
Epo

In [11]:
result = model.predict(test.drop(columns=["order_id"]))

893/893 [==============================] - 1s 1ms/step


In [12]:
result = [r[0] for r in result]

submission = pd.DataFrame({
    "order_id":list(test["order_id"]),
    "late_order":list(result)
})

submission.fillna(0, inplace=True)

submission.to_csv("result.csv", index=False)

In [13]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28563 entries, 0 to 28562
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    28563 non-null  object 
 1   late_order  28563 non-null  float32
dtypes: float32(1), object(1)
memory usage: 334.8+ KB
